Найти решение краевой задачи для одномерного стационарного уравнения теплопроводности
$$\frac{d}{dx}[k(x)\frac{du}{dx}] - q(x)u = -f(x)$$
в 11 равноудаленных точках отрезка $[0, 1]$ с относительной точностью $0.0001$.
Отладку программы произвести на модельной задаче с постоянными коэффициентами.

In [130]:
from numpy import sin, cos, exp, sqrt, linspace, round, array, max, abs
from pandas import DataFrame

Условие. Задача 4.

Краевые условия:
$$u(0) = 1; u(1) = 0$$
\begin{equation*}
 \begin{cases}
    u(x_0 - 0) = u(x_0 + 0) \\
    k(x_0 - 0) u_{x}(x_0 - 0) = k(x_0 + 0) u_{x}(x_0 + 0) 
 \end{cases}
\end{equation*}
$$x > x_0 = 0.525 : k(x) = x; q(x) = e^{-x}; f(x) = x^3$$
$$x < x_0: k(x) = x^2 + 1; q(x) = e^{-x}; f(x) = 1$$

Модельная задача:
$$x_0 = 0.525 : k(x) = k(x_0); q(x) = q(x_0); f(x) = f(x_0)$$

In [131]:
def kb(x):
    return x


# k for i < la
def ka(x):
    return x**2 + 1


# q for i < la
def qa(x):
    return exp(-x)


# q for i > lb
def qb(x):
    return exp(-x)


# f for i > lb
def fb(x):
    return x**3

# f for i < la
def fa(x):
    return 1

In [132]:
#условия
accur = 10**(-4)

u0 = 0
u1 = 1

x0 = 0.525



Аналитическое решение модельной задачи

In [133]:
def cs(l1, l2):
    a11 = exp(-l1*x0)-exp(l1*x0)
    a12 = exp(l2*(2-x0))-exp(l2*x0)
    a21 = ka(x0)*l1*(exp(l1*x0)+exp(-l1*x0))
    a22 = kb(x0)*l2*(exp(l2*(2-x0))+exp(l2*x0))
    ma = fa(x0)/qa(x0)
    mb = fb(x0)/qb(x0)
    b1 = mb-ma+(ma-u0)*exp(l1*x0)-(mb-u1)*exp(l2*(1-x0))
    b2 = ka(x0)*l1*(u0-ma)*exp(l1*x0)+kb(x0)*l2*(u1-mb)*exp(l2*(1-x0))
    c1 = (((u0-ma)*a11-b1)*a22-((u0-ma)*a21-b2)*a12)/(a11*a22-a12*a21)
    c2 = (b1*a22-b2*a12)/(a11*a22-a12*a21)
    c3 = (b2*a11-b1*a21)/(a11*a22-a12*a21)
    c4 = (u1-mb)*exp(l2)-c3*exp(2*l2)
    return c1, c2, c3, c4

def model_u(x, l_1, l_2, c_1, c_2, c_3, c_4):
    if x < x0:
        c1 = c_1
        c2 = c_2
        m = fa(x0)/qa(x0)
        l1 = l_1
    else:
        c1 = c_3
        c2 = c_4
        m = fb(x0)/qb(x0)
        l1 = l_2
    return c1 * exp(l1 * x) + c2 * exp(-l1 * x) + m


Прогонка


In [134]:
def counter_sweep_method(a, b, c, d, la, lb):
    n = len(a)
    alpha = [0] * (n-1)
    alpha[1] = -a[1]/b[1]
    alpha[-1] = -c[-1]/b[-1]
    beta = [0] * (n-1)
    beta[1] = (d[1]-c[1]*u0)/b[1]
    beta[-1] = (d[-1]-c[-1]*u1)/b[-1]
    x = [u0] * n
    x[-1] = u1
    for i in range(2, la):
        alpha[i] = (-a[i]/(c[i]*alpha[i-1]+b[i]))
        beta[i] = ((d[i]-c[i]*beta[i-1])/(b[i]+c[i]*alpha[i-1]))
    for i in reversed(range(lb+1, n-2)):
        alpha[i] = -c[i]/(b[i]+a[i]*alpha[i+1])
        beta[i] = (d[i]-a[i]*beta[i+1])/(b[i]+a[i]*alpha[i+1])
    x[lb] = x[la] = (ka(x0)*beta[la-1]+kb(x0)*beta[lb+1])/(ka(x0)*(1-alpha[la-1])+kb(x0)*(1-alpha[lb+1]))
    for i in reversed(range(1, la)):
        x[i] = alpha[i]*x[i+1]+beta[i]
    for i in range(lb+1, n-1):
        x[i] = alpha[i]*x[i-1]+beta[i]
    return x

def u_model_comp(n):
    a, b, c, d = init_const_abcd(n)
    x = counter_sweep_method(a, b, c, d, la, lb)
    return [x[int(i*(n-1)/10)] for i in range(11)]

Численное решение модельной задачи с заданной степенью точности(Метод встречных прогонок)

In [135]:
def init_const_abcd(n):
    a, b, c, d = [], [], [], []
    for i in range(n):
        a.append(ka(x0) if i < la else kb(x0))
        b.append(-2 * ka(x0) - qa(x0) * h ** 2 if i < la else -2 * kb(x0) - qb(x0) * h ** 2)
        c.append(ka(x0) if i < la else kb(x0))
        d.append(-fa(x0) * h ** 2 if i < la else -fb(x0) * h ** 2)
    return a, b, c, d

def u_model_comp(n):
    a, b, c, d = init_const_abcd(n)
    x = counter_sweep_method(a, b, c, d, la, lb)
    return [x[int(i*(n-1)/10)] for i in range(11)]

def u_var_comp(n):
    a, b, c, d = init_variable_abcd(n)
    x = counter_sweep_method(a, b, c, d, la, lb)
    return [x[int(i*(n-1)/10)] for i in range(11)]


Численное решение задачи с переменными коэффициентами 

In [136]:

def init_variable_abcd(n):
    a, b, c, d = [], [], [], []
    for i in range(n):
        a.append(ka(h * (i + 1. / 2)) if i < la else kb(h * (i + 1. / 2)))
        b.append(-(ka(h * (i + 1. / 2)) + ka(h * (i - 1. / 2)) + qa(h * i) * h ** 2) if i < la else -(
        kb(h * (i + 1. / 2)) + kb(h * (i - 1. / 2)) + qb(h * i) * h ** 2))
        c.append(ka(h * (i - 1. / 2)) if i < la else kb(h * (i - 1. / 2)))
        d.append(-fa(h * i) * h ** 2 if i < la else -fb(h * i) * h ** 2)
    return a, b, c, d

def u_var_comp(n):
    a, b, c, d = init_variable_abcd(n)
    x = counter_sweep_method(a, b, c, d, la, lb)
    return [x[int(i*(n-1)/10)] for i in range(11)]

In [137]:


#разделения отрезка на n частей
L = 11#менять на сдаче

x = round(linspace(0.0, 1.0, L), 7)

h = 1 / L

la = int(x0 / h)
lb = int(round(x0 / h))

rep_x = linspace(0.0, 1.0, 11)
l1 = sqrt(qa(x0)/ka(x0))
l2 = sqrt(qb(x0)/kb(x0))
c1, c2, c3, c4 = cs(l1, l2)

u_analytical = array([model_u(i, l1, l2, c1, c2, c3, c4) for i in rep_x])
u_sweep = array(u_model_comp(L))
u_sweep_tmp = array(u_var_comp(L))

dat = {
    'x' : rep_x,
    'Analytical solve' : u_analytical,
    'Sweep solve' : u_sweep,
    'Error' : u_analytical - u_sweep,
    'Max error' : max(abs(u_analytical - u_sweep)),
    'Sweep tmp solve' : u_sweep_tmp
}
df = DataFrame(data=dat)
df

,x,Analytical solve,Sweep solve,Error,Max error,Sweep tmp solve
0,0.0,-4.440892e-16,0.000000,-4.440892e-16,0.004358,0.000000
1,0.1,8.416356e-02,0.083826,3.378782e-04,0.004358,0.113483
2,0.2,1.608752e-01,0.160440,4.347862e-04,0.004358,0.216327
3,0.3,2.304909e-01,0.230188,3.027394e-04,0.004358,0.307924
4,0.4,2.933336e-01,0.293382,-4.834562e-05,0.004358,0.388111
5,0.5,3.496947e-01,0.350305,-6.105579e-04,0.004358,0.457147
6,0.6,4.574248e-01,0.453067,4.357538e-03,0.004358,0.572728
7,0.7,5.857906e-01,0.582796,2.994698e-03,0.004358,0.698282
8,0.8,7.180042e-01,0.716214,1.789868e-03,0.004358,0.807321
9,0.9,8.555569e-01,0.854778,7.784696e-04,0.004358,0.902079
